## Analyzing CSA's Lessons Learned | Analyse des leçons apprises par l'ASC 
### Step 1: Standardizing the Data | Étape 1 : Normalisation des données 
This notebook translates lessons learned written in either English or French in order to facilitate natural language processing (NLP). These NLP analyses are included in the step 2 and step 3 notebooks (sentiment and topic analysis, respectively).   
This workflow could be adapted to any other spreadsheet or csv. 

Please note that the data bundled with these notebooks is dummy data. It was collected from public CSA documents. These documents are available on the CSA Open Data and Information Portal, specifically: https://donnees-data.asc-csa.gc.ca/dataset/6a854752-14bb-41bc-86c9-1d20b59b79e1 and https://donnees-data.asc-csa.gc.ca/dataset/8b05da69-98c3-4cc9-8eee-ce6f5378dda9. 

Ce cahier traduit les leçons apprises écrites en anglais ou en français afin de faciliter le traitement de langage naturel (NLP). Ces analyses NLP sont incluses dans les cahiers des étapes 2 et 3 (analyse des sentiments et des thèmes, respectivement).   
Ce workflow pourrait être adapté à tout autre tableur ou csv.

Veuillez noter que les données jointes à ces carnets sont des données fictives. Elles ont été recueillies à partir de documents publics de l'ASC. Ces documents sont disponibles sur le portail d'information et de données ouvertes de l'ASC, plus précisément : https://donnees-data.asc-csa.gc.ca/fr/dataset/6a854752-14bb-41bc-86c9-1d20b59b79e1 et https://donnees-data.asc-csa.gc.ca/fr/dataset/8b05da69-98c3-4cc9-8eee-ce6f5378dda9. 

Author/Auteur: N Fee, Canadian Space Agency/Agence spatiale canadienne, 2021-06-18 


In [1]:
import nltk #NLP library
import pandas as pd
from langdetect import detect #determines the language of a text 
from deep_translator import GoogleTranslator #automated translation 


In [2]:
infile = "1_Input/dummy_data.xlsx" #spreadsheet generated by the analyze_lessons.ipynb notebook (scraping the docx files). You could also use any other CSV or spreadsheet. 
outfile = "2_Output/LessonsLearned_step1.xlsx" #output

lesson_col = 'Lesson Learned' #name of column that holds the lessons learned text

In [3]:
#Detects the language of the text. This wrapper makes it easier to apply it to a dataframe.
def detect_lang(row):
    lang = detect(row[lesson_col])
    return lang

#Goes from 1 bilingual column to 1 unlingual column according to the user-defined lang_pref variable ('fr' = French, anything else will be assumed to be English 'en')
def translate_lessons(row,lang_pref):
    to_trans,is_list = split_translate(row[lesson_col]) #important because the translation library can only read 5000 characters at a time
    translated = '' 
    if lang_pref == 'fr':
        if row['Language'] == 'en':
            if is_list == True: #if the text is >5000 characters, it will be stored as a list of <4999 character strings and needs to be converted back to a single string 
                for elem in to_trans: #for each 4999 character chunk
                    trans_list = GoogleTranslator(source='en', target='fr').translate(elem)
                    translated = translated + trans_list #add it back to a single string 
            else:
                translated = GoogleTranslator(source='en', target='fr').translate(to_trans)              
                
        else:
            translated = row[lesson_col] #if the text is already in the preferred language, copy it unchanged. 
    else: #if lang_pref is not French (ie. assumed to be English)
        if row['Language']=='fr':
            if is_list == True:
                for elem in to_trans:
                    trans_list = GoogleTranslator(source='fr', target='en').translate(elem)
                    translated = translated + trans_list
            else:
                translated = GoogleTranslator(source='fr', target='en').translate(to_trans)
        else:
            translated = row[lesson_col]
    return translated        

#Go from 1 string to 1 list of strings, each representing a 4999 character or less chunk of the original string
#Necessary to work around a limitation of the translation library 
def split_translate(to_translate):

    if len(to_translate) > 4999:
        chunks = [to_translate[i:i+4999] for i in range(0, len(to_translate), 4999)]
        is_list = True 
    else:
        chunks = to_translate
        is_list= False
    return chunks, is_list
            

In [4]:
#Read the excel input file into dataframe 
lessons_raw = pd.read_excel(infile)
print(lessons_raw[lesson_col])


0     I am honoured to present the State of the Cana...
1     Les efforts de soutien à l’innovation et de co...
2     There are several differing programs and servi...
3     Encourager le lancement de nouvelles entrepris...
4     Research and development (R&D) expenditures to...
5     The plan proposes a commitment to fund the ini...
6     Le Réseau d’innovation spatial canadien (RISC)...
7     To better reflect the current best practices a...
8     Le plan opérationnel décrit les détails sur le...
9     Le gouvernement du Canada appuie depuis longte...
10    In order to measure the changes taking place i...
11    Parallèlement aux efforts concertés avec l’OCD...
12    Le nombre de travailleurs s’est élevé à 8 200,...
13    Il importe de noter que toutes les information...
14    Over the past five years, total revenues in Qu...
15    The following table reports the percentage of ...
16    En 2013, les universités et les centres de rec...
17    La main-d’œuvre du secteur spatial canadie

In [5]:
#Determine the language of each lesson learned cell and create a new column with a language flag
lessons_raw['Language'] = lessons_raw.apply(lambda row: detect_lang(row), axis=1)
print(lessons_raw['Language'])

0     en
1     fr
2     en
3     fr
4     en
5     en
6     fr
7     en
8     fr
9     fr
10    en
11    fr
12    fr
13    fr
14    en
15    en
16    fr
17    fr
18    fr
Name: Language, dtype: object


In [6]:
#Translate each lesson learned into English and French, storing them in their own dataframe column
lessons_raw['Lessons EN'] = lessons_raw.apply(lambda row: translate_lessons(row,'en'), axis=1)
lessons_raw['Lessons FR'] = lessons_raw.apply(lambda row: translate_lessons(row,'fr'), axis=1)

print(lessons_raw['Lessons FR'])

0     J'ai l'honneur de présenter le rapport sur l'é...
1     Les efforts de soutien à l’innovation et de co...
2     Il existe plusieurs programmes et services dif...
3     Encourager le lancement de nouvelles entrepris...
4     Les dépenses de recherche et développement (R&...
5     Le plan propose un engagement à financer les t...
6     Le Réseau d’innovation spatial canadien (RISC)...
7     Afin de mieux refléter les meilleures pratique...
8     Le plan opérationnel décrit les détails sur le...
9     Le gouvernement du Canada appuie depuis longte...
10    Afin de mesurer les changements qui s'opèrent ...
11    Parallèlement aux efforts concertés avec l’OCD...
12    Le nombre de travailleurs s’est élevé à 8 200,...
13    Il importe de noter que toutes les information...
14    Au cours des cinq dernières années, les revenu...
15    Le tableau suivant présente le pourcentage de ...
16    En 2013, les universités et les centres de rec...
17    La main-d’œuvre du secteur spatial canadie

In [7]:
#Save to an Excel spreadsheet file
lessons_raw.to_excel(outfile,index=False, encoding="utf-8")